In [1]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [2]:
def getSamplingData():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()

        cur.execute("""select samplingfeatureid, samplingfeatureuuid, samplingfeaturecode, samplingfeaturename, 
                       samplingfeaturedescription, elevation_m, elevationdatumcv, samplingfeaturegeotypecv, 
                       samplingfeaturetypecv, st_x(st_astext(featuregeometry)) as longitude, 
                       st_y(st_astext(featuregeometry)) as latitude  from sampling_features""")

        rows = cur.fetchall()
        
        df = pd.DataFrame(columns = ['samplingfeatureid', 'samplingfeatureuuid', 'samplingfeaturecode', 
                                     'samplingfeaturename','samplingfeaturedescription', 'elevation_m', 
                                     'elevationdatumcv', 'samplingfeaturegeotypecv', 'samplingfeaturetypecv', 
                                     'longitude', 'latitude'])

        for row in rows:
            df = df.append({'samplingfeatureid': row[0], 'samplingfeatureuuid': row[1], 'samplingfeaturecode': row[2], 
                            'samplingfeaturename': row[3],'samplingfeaturedescription': row[4], 'elevation_m': row[5], 
                            'elevationdatumcv': row[6], 'samplingfeaturegeotypecv': row[7], 'samplingfeaturetypecv': row[8], 
                            'longitude': row[9], 'latitude': row[10]}, ignore_index = True)

        return df
                          
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
df = getSamplingData()
df.head()

,samplingfeatureid,samplingfeatureuuid,samplingfeaturecode,samplingfeaturename,samplingfeaturedescription,elevation_m,elevationdatumcv,samplingfeaturegeotypecv,samplingfeaturetypecv,longitude,latitude
0,18848,34b63b0c-0224-4cc6-bd85-96ca1d54acfc,GB50_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB50,None,None,None,Specimen,NaN,NaN
1,10001,7af6f32b-7445-4107-8dcd-1181781befb4,CHA,Charlotte Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-80.741571,35.303831
2,10003,9bf42b1e-d7f8-4b51-b7dd-3c8a5d4c90dd,GRV,Greenville Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-77.484817,35.633604
3,10004,ac72da6e-215a-449d-b54c-9b28985eca3c,RTP,Morrisville Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-78.869148,35.843160
4,10005,9d5339b5-df3e-4266-bc4e-ef16ea82cb64,WIL,Wilmington Ambient Atmosphere Sampling Site,...,None,None,Point,Site,-77.863337,34.220339


In [4]:
#outfile = '../ncpfast-db/SampleFeatures.csv'
#df.to_csv(outfile, header=True, index=False)

In [2]:
def getSamplingSiteData():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()

        cur.execute("""SELECT s.samplingfeatureid, s.latitude, ST_y(ST_AsText(f.featuregeometry)) AS lat, s.longitude, 
                       ST_x(St_AsText(f.featuregeometry)) AS lon, f.samplingfeaturecode, f.samplingfeaturename,
                       f.samplingfeaturedescription, f.samplingfeaturegeotypecv, f.samplingfeaturetypecv, s.sitetypecv
                       FROM sites s INNER JOIN sampling_features f ON s.samplingfeatureid = f.samplingfeatureid""")
                       #WHERE f.samplingfeaturecode LIKE 'CHA%'""")
        rows = cur.fetchall()
        #WHERE f.samplingfeaturecode LIKE 'GB%' OR f.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['samplingfeatureid', 'latitude', 'lat', 'longitude', 'lon',
                                     'samplingfeaturecode', 'samplingfeaturename','samplingfeaturedescription', 
                                     'samplingfeaturegeotypecv', 'samplingfeaturetypecv', 'sitetypecv'])

        for row in rows:
            df = df.append({'samplingfeatureid': row[0], 'latitude': row[1], 'lat': row[2], 'longitude': row[3],
                            'lon': row[4], 'samplingfeaturecode': row[5], 'samplingfeaturename': row[6],
                            'samplingfeaturedescription': row[7], 'samplingfeaturegeotypecv': row[8], 
                            'samplingfeaturetypecv': row[9], 'sitetypecv': row[10]}, ignore_index = True)

        return df
                          
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
df = getSamplingSiteData()
df

,samplingfeatureid,latitude,lat,longitude,lon,samplingfeaturecode,samplingfeaturename,samplingfeaturedescription,samplingfeaturegeotypecv,samplingfeaturetypecv,sitetypecv
0,10001,35.303831,35.303831,-80.741571,-80.741571,CHA,Charlotte Ambient Atmosphere Sampling Site,...,Point,Site,Atmosphere
1,10003,35.633604,35.633604,-77.484817,-77.484817,GRV,Greenville Ambient Atmosphere Sampling Site,...,Point,Site,Atmosphere
2,10004,35.843160,35.843160,-78.869148,-78.869148,RTP,Morrisville Ambient Atmosphere Sampling Site,...,Point,Site,Atmosphere
3,10005,34.220339,34.220339,-77.863337,-77.863337,WIL,Wilmington Ambient Atmosphere Sampling Site,...,Point,Site,Atmosphere
4,10006,34.220339,34.220339,-77.863337,-77.863337,Wilmington,Wilmington Atmospheric Deposition Site,...,Point,Site,Atmosphere
...,...,...,...,...,...,...,...,...,...,...,...
275,18756,35.266500,35.266500,-81.184800,-81.184800,S02-18,TWO RIVERS UTILITIES (39066),Mt. Island Lake,Point,Site,Aggregate surface-water-use
276,18764,35.296400,35.296400,-82.340400,-82.340400,S3B-61,"TRYON, TOWN OF (39139)",Colt Creek,Point,Site,Aggregate surface-water-use
277,18825,36.147200,36.147200,-80.421500,-80.421500,2-0,"WINSTON-SALEM, CITY OF (39213)",Yadkin River,Point,Site,Aggregate surface-water-use
278,18836,35.587100,35.587100,-80.462800,-80.462800,S06-22,"FAITH, TOWN OF (66799)",Well #6,Point,Site,Aggregate surface-water-use


In [6]:
def getSamplingSpecimenData():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()

        cur.execute("""SELECT s.samplingfeatureid, ST_y(ST_AsText(f.featuregeometry)) AS lat, 
                       ST_x(St_AsText(f.featuregeometry)) AS lon, f.samplingfeaturecode, f.samplingfeaturename,
                       f.samplingfeaturedescription, f.samplingfeaturegeotypecv, 
                       s.isfieldspecimen,s.specimenmediumcv,s.specimentypecv
                       FROM specimens s INNER JOIN sampling_features f 
                       ON s.samplingfeatureid = f.samplingfeatureid
                       ORDER BY f.samplingfeaturecode""")
                       #WHERE f.samplingfeaturecode LIKE 'CHA%'""")
        rows = cur.fetchall()
        #WHERE f.samplingfeaturecode LIKE 'GB%' OR f.samplingfeaturecode LIKE 'MC%'
        
        df = pd.DataFrame(columns = ['samplingfeatureid', 'lat', 'lon', 'samplingfeaturecode', 
                                     'samplingfeaturename','samplingfeaturedescription', 
                                     'samplingfeaturegeotypecv', 'isfieldspecimen', 'specimenmediumcv', 
                                     'specimentypecv'])

        for row in rows:
            df = df.append({'samplingfeatureid': row[0], 'lat': row[1], 'lon': row[2], 
                            'samplingfeaturecode': row[3], 'samplingfeaturename': row[4],
                            'samplingfeaturedescription': row[5], 'samplingfeaturegeotypecv': row[6], 
                            'isfieldspecimen': row[7], 'specimenmediumcv': row[9], 
                            'specimentypecv': row[9]}, ignore_index = True)

        return df
                          
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [7]:
df = getSamplingSpecimenData()
df

,samplingfeatureid,lat,lon,samplingfeaturecode,samplingfeaturename,samplingfeaturedescription,samplingfeaturegeotypecv,isfieldspecimen,specimenmediumcv,specimentypecv
0,18877,None,None,CHA_Specimen,Site Charlotte Ambient Atmosphere Sampling Sit...,Field specimen collected at site CHA,None,True,Individual sample,Individual sample
1,18273,None,None,Dolorem duis blandit,CHATHAM CO-NORTH (39264) where samplingfeatur...,Sapiente reprehender,None,True,Foliage digestion,Foliage digestion
2,18878,None,None,FAY_Specimen,Site Fayetteville Ambient Atmosphere Sampling ...,Field specimen collected at site FAY,None,True,Individual sample,Individual sample
3,18851,None,None,GB110_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB110,None,True,Individual sample,Individual sample
4,18852,None,None,GB130_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB130,None,True,Individual sample,Individual sample
5,18853,None,None,GB150_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB150,None,True,Individual sample,Individual sample
6,18854,None,None,GB170_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB170,None,True,Individual sample,Individual sample
7,18855,None,None,GB190_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB190,None,True,Individual sample,Individual sample
8,18847,None,None,GB20_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB20,None,True,Individual sample,Individual sample
9,18856,None,None,GB210_Specimen,SiteGeorgia Branch Field Specimen,Field specimen collected at site GB210,None,True,Individual sample,Individual sample


In [ ]:
#outfile = '../ncpfast-db/SiteSampleFeatures.csv'
#df.to_csv(outfile, header=True, index=False)

In [ ]:
samplingfeatureid = list(df['samplingfeatureid'])

In [ ]:
def getFeatureActions(samplingfeatureid):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        rows = []
        for featureid in samplingfeatureid:
            cur.execute("""select featureactionid, actionid, samplingfeatureid 
                           from feature_actions 
                           where samplingfeatureid = %(feature_id)s""",
                       {'feature_id': AsIs(featureid)})
            row = cur.fetchall()
            rows.append(row)
            
        return rows
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [ ]:
feature_actions = getFeatureActions(samplingfeatureid)

In [ ]:
def getActions(feature_actions):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        rows = []
        for feature_action in feature_actions:
            actionid = feature_action[0][1]
            cur.execute("""select actionid, begin_datetime, end_datetime, methodid
                           from actions 
                           where actionid = %(action_id)s""",
                       {'action_id': AsIs(actionid)})        
            row = cur.fetchall()
            rows.append(row)
            
        return rows
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [ ]:
actions = getActions(feature_actions)
actions

In [ ]:
len(actions)

In [ ]:
len(feature_actions)

In [ ]:
len(samplingfeatureid)

In [ ]:
feature_actions